In [1]:
from italian_csv_type_prediction.dataframe_generators import SimpleDatasetGenerator
from italian_csv_type_prediction.models import TypePredictor
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
from multiprocessing import Pool, cpu_count
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [2]:
def dataset_generation(number:int):
    return SimpleDatasetGenerator().build(number, verbose=False)

def _dataset_generation(args):
    return dataset_generation(*args)

def parallel_dataset_generation(number:int):
    processes = min(cpu_count()*5, number)
    with Pool(cpu_count()) as p:
        Xs, ys = list(zip(*tqdm(
            p.imap(_dataset_generation, (
                (number//processes, )
                for _ in range(processes)
            )),
            desc="Creating dataset",
            total=processes,
            leave=False
        )))
    return np.vstack(Xs), np.concatenate(ys)

In [3]:
x_train, y_train = parallel_dataset_generation(1000)
x_test, y_test = parallel_dataset_generation(1000)

In [4]:
model = TypePredictor()

model.fit(x_train, y_train)

In [5]:
y_pred = model._model.predict(x_test)
y_train_pred = model._model.predict(x_train)

In [6]:
accuracy_score(y_test, y_pred), balanced_accuracy_score(y_test, y_pred)

(0.9876062328712472, 0.985784939308558)

In [7]:
accuracy_score(y_train, y_train_pred), balanced_accuracy_score(y_train, y_train_pred)

(0.9992612435624404, 0.9992302330364187)

In [8]:
X, y = SimpleDatasetGenerator().generate_simple_dataframe()

In [9]:
y

,ItalianVAT,CadastreCode,Document,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,Year,...,CountryCode,Name,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean,NumericId
0,ItalianFiscalCode,CadastreCode,Document,NaN,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,Year,...,CountryCode,NaN,Surname,String,NaN,PhoneNumber,NaN,BiologicalSex,NaN,Error
1,ItalianVAT,CadastreCode,Document,Plate,Address,ItalianZIPCode,NaN,NaN,NaN,Year,...,Error,Name,Surname,String,NaN,NaN,Date,BiologicalSex,Boolean,NaN
2,ItalianFiscalCode,CadastreCode,Document,NaN,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,NaN,...,CountryCode,Name,Surname,String,NaN,PhoneNumber,Date,NaN,Boolean,Error
3,NaN,Error,Document,Error,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,Year,...,CountryCode,Name,Surname,String,EMail,NaN,Error,NaN,Boolean,NumericId
4,NaN,CadastreCode,Document,Plate,NaN,NaN,ProvinceCode,Region,Error,Year,...,CountryCode,Name,Surname,NaN,EMail,Error,Date,BiologicalSex,Boolean,NumericId
5,ItalianFiscalCode,CadastreCode,Document,Plate,Address,NaN,ProvinceCode,Region,Municipality,Year,...,CountryCode,Name,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean,NaN
6,ItalianVAT,CadastreCode,Document,NaN,NaN,ItalianZIPCode,ProvinceCode,Region,Municipality,Year,...,NaN,Name,Surname,String,EMail,NaN,Date,BiologicalSex,Boolean,NumericId
7,ItalianFiscalCode,CadastreCode,NaN,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,Year,...,CountryCode,Name,Surname,String,EMail,PhoneNumber,Date,Error,NaN,NumericId
8,ItalianVAT,NaN,Document,Plate,Address,ItalianZIPCode,NaN,Region,NaN,Year,...,NaN,Name,Surname,String,NaN,PhoneNumber,Date,BiologicalSex,Boolean,NaN
9,Error,NaN,Document,Plate,NaN,ItalianZIPCode,ProvinceCode,Region,Error,Year,...,CountryCode,Name,Surname,String,EMail,PhoneNumber,NaN,BiologicalSex,Boolean,NumericId


In [10]:
from collections import Counter

mask = y_test != y_pred

true_labels = model._embedder._encoder.inverse_transform(y_test[mask])
predicted_labels = model._embedder._encoder.inverse_transform(y_pred[mask])

Counter(zip(true_labels, predicted_labels))

Counter({('CountryCode', 'NaN'): 69,
         ('Error', 'NumericId'): 46,
         ('Country', 'Error'): 566,
         ('Error', 'Document'): 148,
         ('Error', 'Name'): 41,
         ('Region', 'String'): 19,
         ('Error', 'String'): 24,
         ('NumericId', 'Error'): 88,
         ('Error', 'Country'): 204,
         ('CountryCode', 'Boolean'): 30,
         ('Error', 'Boolean'): 34,
         ('String', 'Address'): 1392,
         ('Error', 'ItalianFiscalCode'): 22,
         ('Municipality', 'Error'): 19,
         ('Error', 'Municipality'): 18,
         ('Error', 'ItalianZIPCode'): 9,
         ('Error', 'ProvinceCode'): 33,
         ('Error', 'CountryCode'): 25,
         ('NaN', 'CountryCode'): 101,
         ('Address', 'String'): 935,
         ('Error', 'BiologicalSex'): 54,
         ('NaN', 'Error'): 15,
         ('Error', 'ItalianVAT'): 23,
         ('Error', 'Integer'): 17,
         ('BiologicalSex', 'Error'): 53,
         ('Country', 'Region'): 98,
         ('Error', 'Sur